In [ ]:
import json
from glob import glob
from pprint import pprint
from tqdm import tqdm
import inflect

p = inflect.engine()

In [ ]:
with open("./ambiguous_questions.json") as f:
    ambiguous_questions = json.load(f)

with open("./ambiguous_questions_rebuilt.json") as f:
    ambiguous_questions_pr = json.load(f)

In [ ]:
with open("./sceneGraphs/train_sceneGraphs.json") as f:
    sceneGraphs = json.load(f)

In [ ]:
def get_same_name_entities(sceneGraph, target_object_id):
    
    entities = []
    
    target_object_name = sceneGraph['objects'][target_object_id]['name']
    for object_key, object_value in sceneGraph["objects"].items():
        if object_key != target_object_id and object_value['name'] == target_object_name:
            entities.append((object_key, object_value))
            
    return entities

In [ ]:
def make_unique_attribute_question(sceneGraph, target_object_id):
    
    questions = []
    
    target_object = sceneGraph["objects"][target_object_id]
    
    same_entities = get_same_name_entities(sceneGraph, target_object_id)
    
    target_attributes = set(target_object["attributes"])
    
    question_type = "attribute"
    
    if len(same_entities) == 0:
        return questions
    
    # print(target_attributes)
    
    for entity_key, entity_value in same_entities:
        target_attributes = target_attributes - set(entity_value['attributes'])
        
    yes_unique_attributes = target_attributes
    
    # print(target_attributes)
    
    be_verb = "Is" if p.singular_noun(target_object['name']) == False else "Are"
    
    for unique_attribute in yes_unique_attributes:
        
        intermediate_question = f"{be_verb} the {target_object['name']} {unique_attribute}?"
        intermediate_answer = "yes"

        questions.append({'question' : intermediate_question,
                          'answer' : intermediate_answer,
                          'type': question_type,
                          'label' : "positive"})
    
    intersection = None
    for entity_key, entity_value in same_entities:
        
        if intersection is None:
            intersection = set(entity_value['attributes'])
        else:
            intersection = intersection & set(entity_value['attributes'])
            
    if intersection is not None:
        no_unique_attributes = intersection - set(target_object["attributes"])

        # print(no_unique_attributes)
        
        for unique_attribute in no_unique_attributes:
            
            intermediate_question = f"{be_verb} the {target_object['name']} {unique_attribute}?"
            intermediate_answer = "no"

            questions.append({'question' : intermediate_question,
                            'answer' : intermediate_answer,
                            'type': question_type,
                            'label' : "positive"})
        
    return questions

In [ ]:
def relations2set(relations):
    
    values = []
    for relation in relations:
        name = relation['name']
        object = relation['object']
        
        values.append((name, object))
        
    return set(values)

def make_unique_position_question(sceneGraph, unique_relations, target_object_name):
    
    be_verb = "Is" if  p.singular_noun(target_object_name) == False else "Are"
    
    left_count = 0
    right_count = 0
    remove_relations = []
    for unique_relation in unique_relations:
        relation = unique_relation[0]
        object_name = sceneGraph["objects"][unique_relation[1]]['name']
        
        if object_name == target_object_name:
            if "left" in relation:
                left_count += 1
                remove_relations.append(unique_relation)
            elif "right" in relation:
                right_count += 1
                remove_relations.append(unique_relation)
            else:
                pass
    
    for remove_relation in remove_relations:
        unique_relations.remove(remove_relation)
    
    if left_count + right_count > 1:
        object_name = p.plural(target_object_name)
    else:
        object_name = target_object_name
    
    position_relation = ""
    intermediate_answer = "yes"
    question_type = 'relation'
    
    
    if left_count != 0 and right_count == 0:
        position_relation = "on the left of"
        intermediate_question = f"{be_verb} the {target_object_name} {position_relation} the other {object_name}?"
        
    elif left_count == 0 and right_count != 0:
        position_relation = "on the right of"
        intermediate_question = f"{be_verb} the {target_object_name} {position_relation} the other {object_name}?"
    elif left_count != 0 and right_count != 0 and left_count == right_count:
        position_relation  = "in the middle of"
        intermediate_question = f"{be_verb} the {target_object_name} {position_relation} the other {object_name}?"
    else:
        pass
    
    if position_relation == "":
        return None, unique_relations 
    else:
        question = {"question" : intermediate_question,
                    "answer" : intermediate_answer,
                    'type' : question_type,
                    'label' : "positive"}
        return question, unique_relations

def make_unique_relation_question(sceneGraph, target_object_id):
    
    questions = []
    
    target_object = sceneGraph["objects"][target_object_id]
    # pprint(target_object)
    
    same_entities =  get_same_name_entities(sceneGraph, target_object_id)
    
    # pprint(same_entities)
    
    if len(same_entities) == 0:
        return questions
    
    target_relations = relations2set(target_object["relations"])
    
    question_type = "relation"
    
    if len(target_relations) > 0 :
        postion_question, yes_unique_relations = make_unique_position_question(sceneGraph, target_relations, target_object['name'])
    
        if postion_question is not None:
            questions.append(postion_question)

        target_relations = yes_unique_relations
    
    for entity_key, entity_value in same_entities:
        target_relations = target_relations - relations2set(entity_value['relations'])
        
    yes_unique_relations = target_relations
    
    be_verb = "Is" if p.singular_noun(target_object['name']) == False else "Are"
    
    for unique_relation in yes_unique_relations:
            
        relation = unique_relation[0]
        object_name = sceneGraph["objects"][unique_relation[1]]['name']
        
        if p.singular_noun(object_name) == False:
            object_name = p.a(object_name)
        
        if len(get_same_name_entities(sceneGraph, unique_relation[1])) == 0:
        
            intermediate_question = f"{be_verb} the {target_object['name']} {relation} {object_name}?"
            intermediate_answer = "yes"

            questions.append({'question' : intermediate_question,
                            'answer' : intermediate_answer,
                            'type': question_type,
                            'label' : "positive"})
    
    intersection = None
    for entity_key, entity_value in same_entities:
        
        if intersection is None:
            intersection =  relations2set(entity_value['relations'])
        else:
            intersection = intersection & relations2set(entity_value['relations'])
            
    if intersection is None:
        no_unique_relations = set()
    else:
        no_unique_relations = intersection - relations2set(target_object["relations"])
        
        # print(no_unique_relations)
        
    for unique_relation in no_unique_relations:
        
        relation = unique_relation[0]
        object_name = sceneGraph["objects"][unique_relation[1]]['name']
        
        if p.singular_noun(object_name) == False:
            object_name = p.a(object_name)
        
        if len(get_same_name_entities(sceneGraph, unique_relation[1])) == 0:
        
            intermediate_question = f"{be_verb} the {target_object['name']} {relation} {object_name}?"
            intermediate_answer = "no"
            
            questions.append({'question' : intermediate_question,
                            'answer' : intermediate_answer,
                            'type': question_type,
                            'label' : "positive"
                            })
        
    return questions